In [1]:
import os
import tarfile
import glob
import numpy as np
import os, sys, tarfile
from nilearn import image as nimg
from nilearn import plotting as nplot
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
# %matplotlib notebook
from matplotlib import patches, ticker
import matplotlib.pyplot as plt
from nilearn.image import get_data
import numpy as np
from nilearn.image import mean_img, index_img, resample_to_img, get_data
from nilearn import plotting
import time
from nilearn.regions import Parcellations
from nilearn import datasets
import pandas as pd
import glob
import warnings
warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
from nilearn import datasets, plotting
from nilearn._utils import  check_niimg
import numpy as np 
from nilearn._utils.ndimage import largest_connected_component 
from scipy import ndimage
from nilearn.image import resampling

#destrieux = datasets.fetch_atlas_destrieux_2009()

def get_parcellation_coordinates(label_img):
    # grab data and affine
    label_img = check_niimg(label_img)
    label_data = label_img.get_data()
    label_affine = label_img.affine

    # grab number of unique values in 3d image
    label_unique = np.unique(label_data)[1:]

    # grab center of mass from parcellations and dump into coords list
    coord_list = []
    for i, cur_label in enumerate(label_unique):
        cur_img = label_data == cur_label

        #take the largest connected component
        volume = np.asarray(cur_img)
        labels, label_nb = ndimage.label(volume)
        if not label_nb:
            raise ValueError('No non-zero values: no connected components')
        if label_nb == 1:
            volume = volume.astype(np.bool) 

        # get parcellation center of mass
        center_of_mass = ndimage.center_of_mass(volume)
        world_coords = resampling.coord_transform(center_of_mass[0],
                                                  center_of_mass[1],
                                                  center_of_mass[2],
                                                  label_affine)

        coord_list.append((world_coords[0], 
                           world_coords[1], 
                           world_coords[2]))

    return coord_list

In [3]:
Desikan_ROIs=['left cerebral white matter',
'left lateral ventricle','left inferior lateral ventricle','left cerebellum white matter','left cerebellum cortex',
'left thalamus','left caudate','left putamen','left pallidum','3rd ventricle','4th ventricle','brain-stem', 
'left hippocampus','left amygdala', 'unknown','left accumbens area', 'left ventral DC','right cerebral white matter',
'right lateral ventricle','right inferior lateral ventricle', 'right cerebellum white matter',
'right cerebellum cortex','right thalamus','right caudate','right putamen', 
'right pallidum','right hippocampus','right amygdala','right accumbens area', 'right ventral DC','ctx-lh-bankssts', 'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-caudalmiddlefrontal', 'ctx-lh-cuneus',
        'ctx-lh-entorhinal', 'ctx-lh-fusiform', 'ctx-lh-inferiorparietal',
        'ctx-lh-inferiortemporal', 'ctx-lh-isthmuscingulate',
        'ctx-lh-lateraloccipital', 'ctx-lh-lateralorbitofrontal',
        'ctx-lh-lingual', 'ctx-lh-medialorbitofrontal',
        'ctx-lh-middletemporal', 'ctx-lh-parahippocampal',
        'ctx-lh-paracentral', 'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis', 'ctx-lh-parstriangularis',
        'ctx-lh-pericalcarine', 'ctx-lh-postcentral',
        'ctx-lh-posteriorcingulate', 'ctx-lh-precentral',
        'ctx-lh-precuneus', 'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-rostralmiddlefrontal', 'ctx-lh-superiorfrontal',
        'ctx-lh-superiorparietal', 'ctx-lh-superiortemporal',
        'ctx-lh-supramarginal', 'ctx-lh-frontalpole',
        'ctx-lh-temporalpole', 'ctx-lh-transversetemporal',
        'ctx-lh-insula', 'ctx-rh-bankssts',
        'ctx-rh-caudalanteriorcingulate', 'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-cuneus', 'ctx-rh-entorhinal', 'ctx-rh-fusiform',
        'ctx-rh-inferiorparietal', 'ctx-rh-inferiortemporal',
        'ctx-rh-isthmuscingulate', 'ctx-rh-lateraloccipital',
        'ctx-rh-lateralorbitofrontal', 'ctx-rh-lingual',
        'ctx-rh-medialorbitofrontal', 'ctx-rh-middletemporal',
        'ctx-rh-parahippocampal', 'ctx-rh-paracentral',
        'ctx-rh-parsopercularis', 'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis', 'ctx-rh-pericalcarine',
        'ctx-rh-postcentral', 'ctx-rh-posteriorcingulate',
        'ctx-rh-precentral', 'ctx-rh-precuneus',
        'ctx-rh-rostralanteriorcingulate', 'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal', 'ctx-rh-superiorparietal',
        'ctx-rh-superiortemporal', 'ctx-rh-supramarginal',
        'ctx-rh-frontalpole', 'ctx-rh-temporalpole',
        'ctx-rh-transversetemporal', 'ctx-rh-insula']

In [4]:
raw_data_path = '/snl/abcd/raw-data/fmri/baseline/fmriresults01/'
os.chdir(raw_data_path) 

In [7]:
# Segmented data directory (per batch)
seg_data_dir='/nadata/cnl/abcd/data/imaging/smri/interim/synthseg/baseline/'
segmented=[]

for filename in glob.iglob(seg_data_dir + '**/sub-*synthseg.nii', recursive=True):
    segmented.append(filename)


In [8]:
data_list=np.loadtxt('/cnl/abcd/data/tabular/processed/labels/highscores.txt', dtype='str')
data_list

array(['NDARINVD7RR0C8L', 'NDARINVU5GEZR43', 'NDARINV7XER6781', ...,
       'NDARINVZG4U8G7X', 'NDARINVZX24TGXN', 'NDARINVZXKWKTC7'],
      dtype='<U15')

In [9]:
len(data_list)

11868

In [10]:
path='/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/'
dataset=[]
for filename in glob.iglob(path + '**/sub-*.nii', recursive=True):
    dataset.append(filename)

In [11]:
len(dataset)

11526

In [12]:
fMRI=[]
to_seg=[]
for d in data_list:
    fMRI=[s for s in dataset if d in s]
    if len(fMRI)>0:
        for n in range(len(fMRI)):
            to_seg.append(fMRI[n])

In [13]:
len(to_seg)

11517

In [21]:
i=0
missing_rois=[]
seg_error=[]
path='/cnl/abcd/data/imaging/fmri/rsfmri/interim/segmented/baseline/downloads/'

for data in to_seg:
    
    if not os.path.exists(path + data.split('/')[8]+'/cortex_fMRI_segmented_'+data.split('/')[8]+data.split('_')[-2]+'.csv'):

        print(data.split('/')[8], i)
        try:
            #     print('Loading segmented sMRI data')
            sMRI=[s for s in segmented if data.split('/')[8] in s]
            img_seg = nib.load(sMRI[0])

        #     print('Loading corresponding fmri')
            fMRI_img = nib.load(data)
            single_fMRI_img = index_img(fMRI_img, 1)

            confounds=data[:-8]+'motion.tsv'

        #     print('Resampling and masking')
            res_img_seg=resample_to_img(img_seg, single_fMRI_img,interpolation='nearest')
            from nilearn.maskers import NiftiLabelsMasker
            masker = NiftiLabelsMasker(labels_img=res_img_seg, standardize=True)
            try:
                fMRI_seg_ts=masker.fit_transform(fMRI_img,confounds=confounds)
                if np.shape(fMRI_seg_ts)[1]<98:
                    print('missing ROIs!')
                    missing_rois.append(data)
                else:
                    time_series=fMRI_seg_ts[17:,30:]


            #         print('Saving segmented fmri')
                    os.chdir(path)
                    isExist = os.path.exists(data.split('/')[8])
                    if not isExist:
                # Create a new directory because it does not exist
                        os.makedirs(data.split('/')[8])
                    os.chdir(path+data.split('/')[8])
            #       
                #     print('Computing ROIs coordinates')
                    if len(glob.glob('ROIs*'))==0:
                        coord_list=get_parcellation_coordinates(img_seg)
                        cl=pd.DataFrame(coord_list, index=Desikan_ROIs, columns=['x', 'y', 'z'])
                        cl.to_csv('ROIs_centroid_coordinates.csv') 

            #         print('Save cortical ts')
                    ts=pd.DataFrame(time_series)
                    ts.to_csv('cortex_fMRI_segmented_'+data.split('/')[8]+data.split('_')[-2]+'.csv', index=None)  
                    ts_all=pd.DataFrame(fMRI_seg_ts)
            #         print('Save all ts')
                    ts_all.to_csv('fMRI_segmented_'+data.split('/')[8]+data.split('_')[-2]+'.csv', index=None) 

            except:
                print('no confounds file')


            i=i+1
        except:
            seg_error.append(data)
            print('File may be corrupted')
            i=i+1
    else:
        print('Done')

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINV963N6B2W 0
missing ROIs!
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINV4C2YD3TH 1
missing ROIs!
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINVLZHG1ZK6 2
missing ROIs!
Done
Done
Do

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINVX8HAGH4H 19
Done
sub-NDARINVX8HAGH4H 20
sub-NDARINVX8HAGH4H 21
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
D

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINVTE7TYC92 34
missing ROIs!
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Don

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


missing ROIs!
sub-NDARINVZW8ENNPU 60
missing ROIs!
sub-NDARINVZW8ENNPU 61
missing ROIs!
sub-NDARINVZW8ENNPU 62
missing ROIs!
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINVL2RJ9HXF 63
missing ROIs!
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Don

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINVKFUY5YHC 74
missing ROIs!
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Don

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
sub-NDARINVY17ZU37V 82
sub-NDARINVY17ZU37V 83
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done

sub-NDARINVYZ0U1DGC 101
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [15]:
seg_error

['/snl/abcd/raw-data/fmri/baseline/fmriresults01/downloads2/sub-NDARINVFP0ZD9PP/ses-baselineYear1Arm1/func/sub-NDARINVFP0ZD9PP_ses-baselineYear1Arm1_task-rest_run-01_bold.nii',
 '/snl/abcd/raw-data/fmri/baseline/fmriresults01/downloads2/sub-NDARINVUJ86W3MP/ses-baselineYear1Arm1/func/sub-NDARINVUJ86W3MP_ses-baselineYear1Arm1_task-rest_run-01_bold.nii',
 '/snl/abcd/raw-data/fmri/baseline/fmriresults01/downloads2/sub-NDARINV7YA52BU1/ses-baselineYear1Arm1/func/sub-NDARINV7YA52BU1_ses-baselineYear1Arm1_task-rest_run-01_bold.nii']

In [22]:
missing_rois

['/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/sub-NDARINV963N6B2W/ses-baselineYear1Arm1/func/sub-NDARINV963N6B2W_ses-baselineYear1Arm1_task-rest_run-04_bold.nii',
 '/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/sub-NDARINV4C2YD3TH/ses-baselineYear1Arm1/func/sub-NDARINV4C2YD3TH_ses-baselineYear1Arm1_task-rest_run-02_bold.nii',
 '/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/sub-NDARINVLZHG1ZK6/ses-baselineYear1Arm1/func/sub-NDARINVLZHG1ZK6_ses-baselineYear1Arm1_task-rest_run-02_bold.nii',
 '/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/sub-NDARINVU3J6ZW8Z/ses-baselineYear1Arm1/func/sub-NDARINVU3J6ZW8Z_ses-baselineYear1Arm1_task-rest_run-01_bold.nii',
 '/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/sub-NDARINVZ4N65WJT/ses-baselineYear1Arm1/func/sub-NDARINVZ4N65WJT_ses-baselineYear1Arm1_task-rest_run-01_bold.nii',
 '/nadata/snl/abcd/raw-data/fmri/baseline/fmriresults01/sub-NDARINVRMTE592D/ses-baselineYear1Arm1/func/sub-NDARINVRMTE592D_ses-baseline